<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/Exemplos_Chunk_Overlap_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de divisão e sobreposiçao de texto usando Langchain

Cria a divisão de um texto em pedaços(chunks) para submeter a um LLM. Realiza a sobreposição(overlap) de parte do texto para manter a semântica entre os chunks.

Tipos de divisão:
- Token (palavras)
- Recursiva de Caracter
- Token (Tokenizador BERT)

Semelhante a uma janela deslizante.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto: https://chunkerizer.streamlit.app/

Entendendo o chunk e overlap
https://gustavo-espindola.medium.com/chunk-division-and-overlap-understanding-the-process-ade7eae1b2bd

LangChain: como dividir corretamente seus chunks(pedaços):
https://www.youtube.com/watch?v=n0uPzvGTFI0

Estratégias de chucking: https://www.pinecone.io/learn/chunking-strategies/

Como agrupar dados de texto - uma análise comparativa: https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a

# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [1]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [2]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [3]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [4]:
!pip install pypdf==3.16.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.8 MB/s eta 0:00:00


Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

Dependência do WebBaseLoader.

In [5]:
!pip install chromadb==0.4.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 45.

Biblioteca para realizar a divisão por token.

In [6]:
!pip install tiktoken==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [7]:
!pip install langchain==0.0.323

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [8]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1


# 2 - Tipos de divisão

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## Divisor recursivo de caracteres

In [9]:
#Import de bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size =26
chunk_overlap = 4

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
)

In [10]:
# Divisor recursivo do texto
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [11]:
# Dividor de caracter do texto
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)
# Saída - ['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [12]:
# Dividor recursivo de texto
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Divisor de caracter

In [13]:
#Import de bibliotecas
from langchain.text_splitter import CharacterTextSplitter

chunk_size =26
chunk_overlap = 4

c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
)

In [14]:
# Dividor de caracter texto
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m n o p q r s t u v w x y z']

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [15]:
# Character Text Splitter with separator defined
# Divisor de caracter texto com separador definido
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

## Divisor de token

In [16]:
#Import de bibliotecas
from langchain.text_splitter import TokenTextSplitter

chunk_size =26
chunk_overlap = 4

t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
)

In [17]:
# Dividor de token do texto
t_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [18]:
# Dividor de token do texto
text4 = 'ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh ij kl mn op qr st uv wx yz'
t_splitter.split_text(text4)
# Saída - ['ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh',' cd ef gh ij kl mn op qr st uv wx yz']

['ab cd ef gh ij kl mn op qr st uv wx yz ab cd ef gh',
 ' cd ef gh ij kl mn op qr st uv wx yz']

# 3 - Divisão por Token

Quebra palavras ao meio.

## 3.1 - Manipulando documentos Texto

### Cria o texto

In [19]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes

texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

### Divide e sobrepõe o texto

In [20]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 50

# Configura o divisor
text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len([texto])} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1 documentos texto em 0.004087924957275391 segundos!


In [21]:
print(len(chunks))

4


In [22]:
print(chunks)

[Document(page_content='\nNo interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consign', metadata={'start_index': 0}), Document(page_content='io,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode co

### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [23]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 650  qtde token : 108  start_index: 0


No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consign
-----------------------------------------------------------------------
chunk # 1  qtde char : 612  qtde token : 96  start_index: 512

io,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fr

Estatísticas dos chunks

In [24]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 650  qtde token : 108
chunk # 1  qtde char : 612  qtde token : 96
chunk # 2  qtde char : 609  qtde token : 107
chunk # 3  qtde char : 423  qtde token : 70
Maior chunk token: 108
Maior chunk character: 650


## 3.2 - Manipulando Documentos PDF

Gera um documento para cada a página do PDF.

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

### - Link dos PDFs

In [25]:
# As Vítimas Algozes
url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# A Escrava Isaura
url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'

### Download dos pdfs

In [26]:
# Import das bibliotecas
import subprocess

# As Vítimas Algozes

# Arquivo de destino1
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

# Arquivo de destino1
destino2 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url2, "-O", destino2])

0

### Carrega os pdfs

In [27]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

 # Cria o carregador do PDF
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os pdfs

In [28]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 50

# Configura o divisor
text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 649 documentos pdfs em 0.533707857131958 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [29]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i,' Doc:',chunk.metadata['source'],' Pág:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: /content/arquivo2.pdf  Pág: 5  qtde char : 26  qtde token : 12  start_index: 0

t.4  t f c ’ ’.’ ■ ■ %  A

-----------------------------------------------------------------------
chunk # 1  Doc: /content/arquivo2.pdf  Pág: 6  qtde char : 5  qtde token : 3  start_index: 0

■  7

-----------------------------------------------------------------------
chunk # 2  Doc: /content/arquivo2.pdf  Pág: 7  qtde char : 88  qtde token : 38  start_index: 0

-• 1. » M  .• * , * .  ’ m . \  ’ é fl» ^  ’ ’ T * A
/. - ’ a .,.•t , ' ,“ *'4 '  ; *
V

-----------------------------------------------------------------------
chunk # 3  Doc: /content/arquivo2.pdf  Pág: 9  qtde char : 4  qtde token : 2  start_index: 0

5 ,

-----------------------------------------------------------------------
chunk # 4  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 520  qtde token : 107  start_index: 0

OBRAS DO MESMO OUTOR
niIA IA KÃ ES (Bernardo;. — 0 ERinTÁo DE AIUQUEM, ou historia da íun- 
? a ? a f a tT

Estatísticas dos chunks

In [30]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' Doc:',chunk.metadata['source'],' Pág:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: /content/arquivo2.pdf  Pág: 5  qtde char : 26  qtde token : 12
chunk # 1  Doc: /content/arquivo2.pdf  Pág: 6  qtde char : 5  qtde token : 3
chunk # 2  Doc: /content/arquivo2.pdf  Pág: 7  qtde char : 88  qtde token : 38
chunk # 3  Doc: /content/arquivo2.pdf  Pág: 9  qtde char : 4  qtde token : 2
chunk # 4  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 520  qtde token : 107
chunk # 5  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 615  qtde token : 100
chunk # 6  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 797  qtde token : 103
chunk # 7  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 901  qtde token : 95
chunk # 8  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 840  qtde token : 90
chunk # 9  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 659  qtde token : 97
chunk # 10  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 136  qtde token : 23
chunk # 11  Doc: /content/arquivo2.pdf  Pág: 14  qtde char : 463  qtde token : 72
chunk # 12  Doc: /content/arquivo2.

## 3.3 - Manipulando Documentos HTML

Gera um único documento para a página HTML.

https://python.langchain.com/docs/integrations/document_loaders/web_base

### Links dos HTML

In [31]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [32]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [33]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import TokenTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = TokenTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 2 documentos htmls em 0.3320791721343994 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [34]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1199  qtde token : 165

















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto 

Estatísticas dos chunks

In [35]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1199  qtde token : 165
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1214  qtde token : 203
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1227  qtde token : 201
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1250  qtde token : 196
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1246  qtde token : 186
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1251  qtde token : 198
chunk # 6  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1187  qtde token : 187
chunk # 7  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=downloa

# 4 - Divisão Recursiva de Caracter

## 4.1 - Manipulando documentos Texto

### Cria o texto

In [36]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes


texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

### Divide e sobrepõe o texto

In [37]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 50

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len([texto])} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1 documentos texto em 0.011907577514648438 segundos!


In [38]:
print(len(chunks))

2


In [39]:
print(chunks)

[Document(page_content='No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode conhecer por suas feições e modos\ncaracterísticos, se nos é lícito dizer assim: uma se parece com todas e não há\nhipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o\ncará\

### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [40]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  qtde char : 1261  qtde token : 204  start_index: 1

No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, 

Estatísticas dos chunks

In [41]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  qtde char : 1261  qtde token : 204
chunk # 1  qtde char : 883  qtde token : 150
Maior chunk token: 204
Maior chunk character: 1261


## 4.2 - Manipulando Documentos PDF

Gera um documento para cada a página do PDF.

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

### - Link dos PDFs

In [42]:
# As Vítimas Algozes
url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# A Escrava Isaura
url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'

### Download dos pdfs

Usando wget

In [43]:
# Import das bibliotecas
import subprocess

# Arquivo de destino1
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

# Arquivo de destino1
destino2 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino2])

0

Ou com usando o curl

In [44]:
# #Import das bibliotecas
# import subprocess

# # Arquivo de destino1
# destino1 = 'arquivo1.pdf'

# # Executa o comando wget no prompt
# subprocess.call(["wget", url1, "-O", destino1])

# # Arquivo de destino1
# destino2 = 'arquivo2.pdf'

# # Executa o comando curl no prompt
# subprocess.call(["curl", "-o", destino, arquivo])

### Carrega os pdfs

In [45]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# Define o diretório
diretorio = '/content'

 # Cria o carregador do PDF
carregador = PyPDFDirectoryLoader(diretorio)

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os pdfs

In [46]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 250
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 706 documentos pdfs em 0.07575678825378418 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [47]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i,' Doc:',chunk.metadata['source'],' Pág:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: /content/arquivo2.pdf  Pág: 0  qtde char : 375  qtde token : 46  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO
-----------------------------------------------------------------------
chunk # 1  Doc: /content/arquivo2.pdf  Pág: 1  qtde char : 78  qtde token : 13  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin
-----------------------------------------------------------------------
chunk # 2  Doc: /content/arquivo2.pdf  Pág: 2  qtde char : 19  qtde token : 2  start_index: 0

AS VICTIMAS-ALGOZES
-----------------------------------------------------------------------
chunk # 3  Doc: /cont

Estatísticas dos chunks

In [48]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    tokens = chunk.page_content.split(" ")
    print('chunk #',i,' Doc:',chunk.metadata['source'],' Pág:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: /content/arquivo2.pdf  Pág: 0  qtde char : 375  qtde token : 46  start_index: 0
chunk # 1  Doc: /content/arquivo2.pdf  Pág: 1  qtde char : 78  qtde token : 13  start_index: 0
chunk # 2  Doc: /content/arquivo2.pdf  Pág: 2  qtde char : 19  qtde token : 2  start_index: 0
chunk # 3  Doc: /content/arquivo2.pdf  Pág: 4  qtde char : 154  qtde token : 25  start_index: 0
chunk # 4  Doc: /content/arquivo2.pdf  Pág: 6  qtde char : 589  qtde token : 94  start_index: 0
chunk # 5  Doc: /content/arquivo2.pdf  Pág: 7  qtde char : 875  qtde token : 139  start_index: 0
chunk # 6  Doc: /content/arquivo2.pdf  Pág: 8  qtde char : 1047  qtde token : 163  start_index: 0
chunk # 7  Doc: /content/arquivo2.pdf  Pág: 9  qtde char : 1039  qtde token : 162  start_index: 0
chunk # 8  Doc: /content/arquivo2.pdf  Pág: 10  qtde char : 997  qtde token : 166  start_index: 0
chunk # 9  Doc: /content/arquivo2.pdf  Pág: 11  qtde char : 834  qtde token : 136  start_index: 0
chunk # 10  Doc: /content/arquivo2

## 4.3 - Manipulando Documentos HTML

Gera um único documento para a página HTML.

https://python.langchain.com/docs/integrations/document_loaders/web_base

### Links dos HTML

In [49]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [50]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [51]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    # length_function = len, # Usa o comprimento do texto em caracteres como medida de tamanho
    length_function = lambda x: len(x.split(" ")), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 2 documentos htmls em 0.12105274200439453 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [52]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2516  qtde token : 401

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que n

Estatísticas dos chunks

In [53]:
maior_chunk_token = 0
maior_chunk_character = 0
for i, chunk in enumerate(chunks):
    # Divide os tokens pelo espaço em branco
    tokens = chunk.page_content.split(" ")
    print('chunk #',i, ' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens) )
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2516  qtde token : 401
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2739  qtde token : 419
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2615  qtde token : 404
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2626  qtde token : 406
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2620  qtde token : 403
chunk # 6  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2637  qtde token : 407
chunk # 7  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&

# 5 - Divisão Recursiva de Caracter e Tokenizador

## 5.1 - Carregando o LM



### Nome do LM BERT

In [54]:
# Modelos em Inglês
#nome_modelo_bert = "bert-large-cased"
#nome_modelo_bert = "bert-base-cased"

# Modelos em Português
nome_modelo_bert = "neuralmind/bert-large-portuguese-cased"
#nome_modelo_bert = "neuralmind/bert-base-portuguese-cased"

### Carregando o LM BERT



Carrega o tokenizador do bert

In [55]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo_bert + ' da comunidade...')

tokenizer_bert = AutoTokenizer.from_pretrained(nome_modelo_bert)

Carregando o tokenizador neuralmind/bert-large-portuguese-cased da comunidade...


Tamanho do vocabulário BERT

In [56]:
print(len(tokenizer_bert))

29794


## 5.2 - Divisão com from_hugginface_tokenizer

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

### Links dos HTML

In [57]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [58]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [59]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 600
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer_bert,
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 2 documentos htmls em 8.069501161575317 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [60]:
for i, chunk in enumerate(chunks):
  if i < 10:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    print('    Tokens bert:', len(tokens_bert), ' Tokens:' , tokens_bert)
    # Documento PDF
    # print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print('       ', chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
    Tokens bert: 50  Tokens: ['As', 'vítimas', 'algo', '##zes', '-', 'Joaquim', 'Manuel', 'de', 'Macedo', 'Fonte', ':', 'Biblioteca', 'Digital', 'de', 'Literatura', 'de', 'Países', 'Lus', '##ófon', '##os', 'L', '##IT', '##ER', '##AT', '##UR', '##A', 'BR', '##AS', '##IL', '##EI', '##RA', 'Tex', '##tos', 'literários', 'em', 'meio', 'eletrônico', 'As', 'Ví', '##timas', '-', 'Al', '##go', '##zes', ',', 'de', 'Joaquim', 'Manuel', 'de', 'Macedo']

        As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=1

Estatísticas dos chunks

In [61]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    #print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert),' start_index:', chunk.metadata.get('start_index') )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1742  qtde token : 276  qtde token bert: 452  start_index: 235
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1891  qtde token : 310  qtde token bert: 481  start_index: 1744
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1927  qtde token : 295  qtde token bert: 484  start_index: 3332
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1850  qtde token : 287  qtde token bert: 470  start_index: 4977
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1857  qtde token : 288  qtde token ber

## 5.3 - Divisão com len(tokenizer)

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

### Links dos HTML

In [62]:
# As Vítimas Algozes
url1 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# A Escrava Isaura
url2 = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92389"

### Carrega os HTMLs

Se ocorrer o **erro** abaixo, instale o Chromedb. Alguma dependência do pacote resolve o problema.

---------------------------------------------------------------------------
IncompleteRead                            Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/urllib3/response.py in _error_catcher(self)
    709             try:
--> 710                 yield
    711

15 frames
IncompleteRead: IncompleteRead(821958 bytes read, 2240625 more expected)

The above exception was the direct cause of the following exception:

ProtocolError                             Traceback (most recent call last)
ProtocolError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

During handling of the above exception, another exception occurred:

ChunkedEncodingError                      Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/requests/models.py in generate()
    816                     yield from self.raw.stream(chunk_size, decode_content=True)
    817                 except ProtocolError as e:
--> 818                     raise ChunkedEncodingError(e)
    819                 except DecodeError as e:
    820                     raise ContentDecodingError(e)

ChunkedEncodingError: ('Connection broken: IncompleteRead(821958 bytes read, 2240625 more expected)', IncompleteRead(821958 bytes read, 2240625 more expected))

In [63]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

# Cria o carregador da página
carregador = WebBaseLoader([url1, url2])
# carregador.requests_kwargs = {'verify':False}

# Carrega os documentos
documentos = carregador.load()

### Divide e sobrepõe os HTMLs

In [64]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 600
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = lambda x: len(tokenizer_bert.tokenize(x)), # Usa a quantidade de tokens separados por espaço em branco como medida de tamanho
    add_start_index = True, # Adiciona o índice de início do chunk no documento
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos htmls em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 2 documentos htmls em 4.820803165435791 segundos!


### Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [65]:
for i, chunk in enumerate(chunks):
  if i < 10:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    print('    Tokens bert:', len(tokens_bert), ' Tokens:' , tokens_bert)
    # Documento PDF
    # print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print('       ', chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
    Tokens bert: 50  Tokens: ['As', 'vítimas', 'algo', '##zes', '-', 'Joaquim', 'Manuel', 'de', 'Macedo', 'Fonte', ':', 'Biblioteca', 'Digital', 'de', 'Literatura', 'de', 'Países', 'Lus', '##ófon', '##os', 'L', '##IT', '##ER', '##AT', '##UR', '##A', 'BR', '##AS', '##IL', '##EI', '##RA', 'Tex', '##tos', 'literários', 'em', 'meio', 'eletrônico', 'As', 'Ví', '##timas', '-', 'Al', '##go', '##zes', ',', 'de', 'Joaquim', 'Manuel', 'de', 'Macedo']

        As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=1

Estatísticas dos chunks

In [66]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    #print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert),' start_index:', chunk.metadata.get('start_index') )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2290  qtde token : 364  qtde token bert: 583  start_index: 235
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2361  qtde token : 364  qtde token bert: 588  start_index: 2158
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2334  qtde token : 355  qtde token bert: 597  start_index: 4130
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2257  qtde token : 349  qtde token bert: 586  start_index: 6170
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 2245  qtde token : 346  qtde token ber